In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler,MultiLabelBinarizer
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score, pairwise_distances
import scipy.cluster.hierarchy as sch





In [2]:
df = pd.read_csv("/media/danlof/dan_files/data_science_codes/career_recommender_system/data/survey_results_public.csv")

In [3]:
df.head()

,ResponseId,Q120,MainBranch,Age,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,ProfessionalTech,Industry,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,I agree,None of these,18-24 years old,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I agree,I am a developer by profession,25-34 years old,"Employed, full-time",Remote,Hobby;Contribute to open-source projects;Boots...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;Friend or fam...,Formal documentation provided by the owner of ...,...,1-2 times a week,10+ times a week,Never,15-30 minutes a day,15-30 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",Appropriate in length,Easy,285000.0
2,3,I agree,I am a developer by profession,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Professional development or self-paced l...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Formal documentation provided by the owner of ...,...,6-10 times a week,6-10 times a week,3-5 times a week,30-60 minutes a day,30-60 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",Appropriate in length,Easy,250000.0
3,4,I agree,I am a developer by profession,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Colleague;Friend or family member;Other online...,Formal documentation provided by the owner of ...,...,1-2 times a week,10+ times a week,1-2 times a week,15-30 minutes a day,30-60 minutes a day,Automated testing;Continuous integration (CI) ...,NaN,Appropriate in length,Easy,156000.0
4,5,I agree,I am a developer by profession,25-34 years old,"Employed, full-time;Independent contractor, fr...",Remote,Hobby;Contribute to open-source projects;Profe...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Online Courses or Certi...,Formal documentation provided by the owner of ...,...,1-2 times a week,1-2 times a week,3-5 times a week,60-120 minutes a day,30-60 minutes a day,Microservices;Automated testing;Observability ...,Other,Appropriate in length,Neither easy nor difficult,23456.0


In [4]:
df.columns

Index(['ResponseId', 'Q120', 'MainBranch', 'Age', 'Employment', 'RemoteWork',
       'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline',
       'LearnCodeCoursesCert', 'YearsCode', 'YearsCodePro', 'DevType',
       'OrgSize', 'PurchaseInfluence', 'TechList', 'BuyNewTool', 'Country',
       'Currency', 'CompTotal', 'LanguageHaveWorkedWith',
       'LanguageWantToWorkWith', 'DatabaseHaveWorkedWith',
       'DatabaseWantToWorkWith', 'PlatformHaveWorkedWith',
       'PlatformWantToWorkWith', 'WebframeHaveWorkedWith',
       'WebframeWantToWorkWith', 'MiscTechHaveWorkedWith',
       'MiscTechWantToWorkWith', 'ToolsTechHaveWorkedWith',
       'ToolsTechWantToWorkWith', 'NEWCollabToolsHaveWorkedWith',
       'NEWCollabToolsWantToWorkWith', 'OpSysPersonal use',
       'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWith',
       'OfficeStackAsyncWantToWorkWith', 'OfficeStackSyncHaveWorkedWith',
       'OfficeStackSyncWantToWorkWith', 'AISearchHaveWorkedWith',
       'AISearchWan

### Data preprocessing 

- Enccoding categorical features and scaling numerical features 

In [5]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89184 entries, 0 to 89183
Data columns (total 84 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ResponseId                           89184 non-null  int64  
 1   Q120                                 89184 non-null  object 
 2   MainBranch                           89184 non-null  object 
 3   Age                                  89184 non-null  object 
 4   Employment                           87898 non-null  object 
 5   RemoteWork                           73810 non-null  object 
 6   CodingActivities                     73764 non-null  object 
 7   EdLevel                              87973 non-null  object 
 8   LearnCode                            87663 non-null  object 
 9   LearnCodeOnline                      70084 non-null  object 
 10  LearnCodeCoursesCert                 37076 non-null  object 
 11  YearsCode                   

Since we want to cluster relevant information to guide future devs and IT professionals ; these are some of the clusters are relevent.

### Job profiles and roles
- `DevType`: This column represents the type of developer or IT professional the respondent identifies as, which is crucial for understanding job profiles.
- `MainBranch`: Indicates the primary focus of the respondent's work (e.g., software development, data science), which is essential for categorizing job profiles.

### Skills and Technologies
- `LanguageHaveWorkedWith`: Lists programming languages the respondent has experience with, which is key for matching job profiles to skills
- `DatabaseHaveWorkedWith`: Similar to programming languages, these columns focus on database technologies
- `PlatformHaveWorkedWith`: Platforms (e.g., cloud services) the respondent has experience with or is interested in.
- `ToolsTechHaveWorkedWith`: Tools and technologies used in professional work.
- `AISearchHaveWorkedWith`: AI search technologies the respondent has experience with or is interested in.
- `AIDevHaveWorkedWith`: AI development tools the respondent has used or wants to learn.

### Experience and Education
- `YearsCode`: Total years of coding experience, which helps in gauging expertise levels.
- `YearsCodePro`: Professional coding experience, important for understanding skill maturity.
- `EdLevel`: The respondent’s education level, which could correlate with job profiles and career paths.
- `LearnCodeCoursesCert`: Information on whether the respondent has taken courses or certifications, relevant for students seeking similar paths.

### Technology Adoption and Usage
- `ProfessionalTech`: Technologies used professionally, relevant for matching skills to job profiles.
- `Industry`: The industry the respondent works in, which might influence the type of technologies used and required skills.



In [6]:
# picking relevant columns 

columns = ['DevType','MainBranch', 'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 
           'PlatformHaveWorkedWith','ToolsTechHaveWorkedWith','AISearchHaveWorkedWith' ,
           'AIDevHaveWorkedWith','YearsCodePro','YearsCode','EdLevel','LearnCodeCoursesCert','ProfessionalTech',
           'Industry']

In [7]:
df = df[columns]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89184 entries, 0 to 89183
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   DevType                  76872 non-null  object
 1   MainBranch               89184 non-null  object
 2   LanguageHaveWorkedWith   87140 non-null  object
 3   DatabaseHaveWorkedWith   73435 non-null  object
 4   PlatformHaveWorkedWith   63628 non-null  object
 5   ToolsTechHaveWorkedWith  77884 non-null  object
 6   AISearchHaveWorkedWith   56328 non-null  object
 7   AIDevHaveWorkedWith      25904 non-null  object
 8   YearsCodePro             66136 non-null  object
 9   YearsCode                87435 non-null  object
 10  EdLevel                  87973 non-null  object
 11  LearnCodeCoursesCert     37076 non-null  object
 12  ProfessionalTech         41783 non-null  object
 13  Industry                 36774 non-null  object
dtypes: object(14)
memory usage: 9.5+ MB


In [8]:
# taking care of missing values 
df = df.fillna('')


In [9]:
df.tail()

,DevType,MainBranch,LanguageHaveWorkedWith,DatabaseHaveWorkedWith,PlatformHaveWorkedWith,ToolsTechHaveWorkedWith,AISearchHaveWorkedWith,AIDevHaveWorkedWith,YearsCodePro,YearsCode,EdLevel,LearnCodeCoursesCert,ProfessionalTech,Industry
89179,"Developer, front-end",I am a developer by profession,HTML/CSS;Java;JavaScript;SQL;TypeScript,MongoDB;MySQL;PostgreSQL;SQLite,Digital Ocean;Firebase;Google Cloud;Heroku;Vercel,Chocolatey;CMake;Docker;Maven (build tool);npm...,ChatGPT,Whispr AI,5,20,"Associate degree (A.A., A.S., etc.)",Udemy,,
89180,"Developer, back-end",I am a developer by profession,Dart;Java;Python;SQL,PostgreSQL,,Maven (build tool);Pip,ChatGPT,,,5,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",,,
89181,,I code primarily as a hobby,Assembly;Bash/Shell (all shells);C;C#;Python;R...,SQLite,Amazon Web Services (AWS),Cargo,ChatGPT;Quora Poe,,,10,Something else,Codecademy;Coursera,,
89182,System administrator,I am a developer by profession,Bash/Shell (all shells);C#;HTML/CSS;Java;JavaS...,MariaDB;Microsoft SQL Server;MongoDB;MySQL;Red...,Amazon Web Services (AWS);Cloudflare;Google Cl...,Docker;Kubernetes;npm;Podman;Vite,Bing AI;ChatGPT;WolframAlpha,,,3,"Secondary school (e.g. American high school, G...",Udemy,,
89183,"Developer, full-stack",I am a developer by profession,C#;Go;JavaScript;SQL;TypeScript,Microsoft SQL Server;Redis;SQLite,Hetzner,Docker;npm;NuGet;pnpm;Vite;Webpack;Yarn,ChatGPT,GitHub Copilot,12,17,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Codecademy;Pluralsight;Coursera,DevOps function;Developer portal or other cent...,"Information Services, IT, Software Development..."


- Apply MultilabelBinarizer for multiple category columns 

In [10]:
# checking the categories in each column
df['AIDevHaveWorkedWith'].unique()

array(['', 'GitHub Copilot', 'GitHub Copilot;Tabnine',
       'AWS CodeWhisperer;GitHub Copilot;Tabnine', 'Adrenaline',
       'Tabnine', 'Replit Ghostwriter',
       'AWS CodeWhisperer;GitHub Copilot',
       'AWS CodeWhisperer;GitHub Copilot;Synk Code;Tabnine',
       'GitHub Copilot;Mintlify', 'Whispr AI', 'Mintlify;Tabnine',
       'GitHub Copilot;Synk Code;Tabnine', 'AWS CodeWhisperer',
       'GitHub Copilot;Whispr AI', 'GitHub Copilot;Rubber Duck.AI',
       'GitHub Copilot;Tabnine;Whispr AI', 'Synk Code',
       'GitHub Copilot;Replit Ghostwriter', 'Replit Ghostwriter;Tabnine',
       'GitHub Copilot;Synk Code', 'AWS CodeWhisperer;Whispr AI',
       'AWS CodeWhisperer;Mintlify', 'Rubber Duck.AI;Tabnine',
       'AWS CodeWhisperer;GitHub Copilot;Whispr AI',
       'Adrenaline;Synk Code;Whispr AI', 'AWS CodeWhisperer;Tabnine',
       'AWS CodeWhisperer;Replit Ghostwriter', 'Adrenaline;Tabnine',
       'GitHub Copilot;Replit Ghostwriter;Tabnine',
       'AWS CodeWhisperer;GitHub C

In [11]:
df['EdLevel'].unique()
## this featues does not contain multiple categories in it

array(['', 'Bachelor’s degree (B.A., B.S., B.Eng., etc.)',
       'Some college/university study without earning a degree',
       'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)',
       'Primary/elementary school',
       'Professional degree (JD, MD, Ph.D, Ed.D, etc.)',
       'Associate degree (A.A., A.S., etc.)',
       'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
       'Something else'], dtype=object)

In [12]:
df['ProfessionalTech'].unique()

array(['',
       'DevOps function;Microservices;Automated testing;Observability tools',
       'DevOps function;Microservices;Automated testing;Observability tools;Innersource initiative;Developer portal or other central places to find tools/services;Continuous integration (CI) and (more often) continuous delivery',
       'Automated testing;Continuous integration (CI) and (more often) continuous delivery',
       'Microservices;Automated testing;Observability tools;Continuous integration (CI) and (more often) continuous delivery',
       'DevOps function;Microservices;Observability tools;Continuous integration (CI) and (more often) continuous delivery',
       'Microservices;Automated testing;Continuous integration (CI) and (more often) continuous delivery',
       'None of these',
       'Automated testing;Observability tools;Continuous integration (CI) and (more often) continuous delivery',
       'Microservices;Observability tools;Developer portal or other central places to find t

In [13]:
df['Industry'].unique()

array(['',
       'Information Services, IT, Software Development, or other Technology',
       'Other', 'Financial Services',
       'Manufacturing, Transportation, or Supply Chain',
       'Retail and Consumer Services', 'Higher Education',
       'Legal Services', 'Insurance', 'Healthcare', 'Oil & Gas',
       'Wholesale', 'Advertising Services'], dtype=object)

In [14]:
df['ProfessionalTech'].unique()

array(['',
       'DevOps function;Microservices;Automated testing;Observability tools',
       'DevOps function;Microservices;Automated testing;Observability tools;Innersource initiative;Developer portal or other central places to find tools/services;Continuous integration (CI) and (more often) continuous delivery',
       'Automated testing;Continuous integration (CI) and (more often) continuous delivery',
       'Microservices;Automated testing;Observability tools;Continuous integration (CI) and (more often) continuous delivery',
       'DevOps function;Microservices;Observability tools;Continuous integration (CI) and (more often) continuous delivery',
       'Microservices;Automated testing;Continuous integration (CI) and (more often) continuous delivery',
       'None of these',
       'Automated testing;Observability tools;Continuous integration (CI) and (more often) continuous delivery',
       'Microservices;Observability tools;Developer portal or other central places to find t

In [15]:
df = df.drop(df.index[0])

In [16]:
mlb = MultiLabelBinarizer()

# features with multiple values 

multi_category_features = ['LanguageHaveWorkedWith','DatabaseHaveWorkedWith','PlatformHaveWorkedWith',
                          'ToolsTechHaveWorkedWith','AISearchHaveWorkedWith','LearnCodeCoursesCert','ProfessionalTech',
                          'AIDevHaveWorkedWith','ProfessionalTech']


- Splitting the semicolon-separated values and using multilabelbinarizer to encode them


In [17]:
for col in multi_category_features:
    # Split the strings 
    df[col] = df[col].apply(lambda x: x.split(';') if isinstance(x, str) else x)

for col in multi_category_features:
    mlb_output = mlb.fit_transform(df[col])
    mlb_df = pd.DataFrame(mlb_output,columns=[f"{col}_{cls}" for cls in mlb.classes_])
    df = df.join(mlb_df, rsuffix=f"_{col}") 

In [ ]:
# onehot encoding 